In [49]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import joblib
import pandas as pd

df = pd.read_csv("dataset/student-mat.csv", sep=';')
df.head(2)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6


In [51]:
df.shape

(395, 33)

In [52]:
# List of columns to remove based on your request
columns_to_remove = ['famrel', 'school', 'sex', 'reason', 'nursery', 'higher', 'freetime', 
                     'goout', 'Dalc', 'Walc', 'famsup', 'schoolsup', 'Mjob', 'Fjob', 
                     'Fedu', 'Medu', 'famsize']
df_cleaned_with_drop = df.drop(columns=columns_to_remove)
# List of columns to keep based on your selection
columns_to_keep = ['age', 'address', 'Pstatus', 'guardian', 'traveltime', 'studytime', 'failures',
                   'paid', 'activities', 'internet', 'romantic', 'health', 'absences', 
                   'G1', 'G2', 'G3']

# Select the columns to keep
df_cleaned = df[columns_to_keep]

In [54]:
# Define features and target
X = df_cleaned.drop(columns=["G3"])  # Features
y = df_cleaned["G3"]  # Target (final grade)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Binary encoding for binary categorical features
binary_features = ['address', 'Pstatus', 'paid', 'activities', 'internet', 'romantic']
X_train[binary_features] = X_train[binary_features].apply(lambda x: x.map({"yes": 1, "no": 0, "U": 1, "R": 0, "T": 1, "A": 0}))
X_test[binary_features] = X_test[binary_features].apply(lambda x: x.map({"yes": 1, "no": 0, "U": 1, "R": 0, "T": 1, "A": 0}))


# Scaling numerical features
numerical_features = ['age', 'traveltime', 'studytime', 'failures', 'health', 'absences', 'G1', 'G2']


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(), ["guardian"])
    ]
)

# Create a pipeline with preprocessing and model
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])
# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calculate R-squared (R²) score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

# Save the model
joblib.dump(model, "student_performance_model.pkl")

Mean Squared Error: 3.2689797468354436
R² Score: 0.8405768246178852


['student_performance_model.pkl']